In [5]:
! pip install -q kaggle

from google.colab import files

# import your kaggle credentials in kaggle.json file sa explained in:
# https://www.kaggle.com/general/74235
files.upload()

{}

In [6]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

# check if kaggle API works
! kaggle datasets list

mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                            title                                               size  lastUpdated          downloadCount  
-------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
arashnic/hr-analytics-job-change-of-data-scientists            HR Analytics: Job Change of Data Scientists        295KB  2020-12-07 00:25:10           1144  
babyoda/access-to-computers-from-home-oecd                     Access to Computers From Home OECD                   3KB  2020-12-04 10:50:09             95  
shashwatwork/impact-of-covid19-pandemic-on-the-global-economy  Impact of Covid-19 Pandemic on the Global Economy    1MB  2020-11-29 14:16:30            566  
emmanuelleai/world-marathons-majors                            World Marathons Majors                               7KB  2020-12-06 19:24:28          

In [10]:
! kaggle datasets download jerzydziewierz/bee-vs-wasp --unzip

 97% 543M/559M [00:02<00:00, 200MB/s]
100% 559M/559M [00:03<00:00, 193MB/s]
